In [ ]:
import pysam
import pandas as pd

In [4]:
gtf_ile = pysam.TabixFile('/sbgenomics/project-files/chr20.sorted.gtf.gz')

In [5]:
#Collect all records of FKBP1A gene?
FKBP1A = []
for gtf in gtf_ile.fetch("20", parser=pysam.asGTF()):
    if gtf.gene_name == 'FKBP1A':
        FKBP1A.append(gtf)

In [6]:
#Collect all exone IDs of FKBP1A exones?
exon_ids = []
for gtf in FKBP1A:
    if gtf.feature == 'exon':
        exon_ids.append(gtf.exon_id)

In [7]:
#Count the exones with unique IDs
len(set(exon_ids))

30

In [17]:
df_list = []
for gtf in gtf_ile.fetch("20", parser=pysam.asGTF()):
    if (gtf.feature == 'start_codon' or gtf.feature == 'CDS' or gtf.feature == 'stop_codon') and gtf.transcript_id == 'ENSE00001872186':
        print()
        df_list.append({'transcript_id': gtf.transcript_id, 'feature': gtf.feature, 
                        'start': gtf.start, 'end': gtf.end})

In [15]:
for gtf in FKBP1A:
    if (gtf.feature == 'start_codon' or gtf.feature == 'CDS' or gtf.feature == 'stop_codon' and gtf.transcript_id == 'ENSE00001872186':
        print({'transcript_id': gtf.transcript_id, 'feature': gtf.feature, 
                        'start': gtf.start, 'end': gtf.end})

AttributeError: 'GTFProxy' has no attribute 'transcript_id'

In [9]:
df = pd.DataFrame(df_list, columns= ['transcript_id', 'feature', 'start', 'end'])

In [10]:
df_ENST00000618612 = df[df['transcript_id'] == 'ENST00000618612'].sort_values(by=['transcript_id', 
                                                                                  'start'], ascending=False)

In [ ]:
df_ENST00000618612

In [ ]:
start = []
end = []
for index, row in df_ENST00000618612.iterrows():
    if row['feature'] == 'CDS':
        start.append(row['start'])
        end.append(row['end'])

In [ ]:
start, end

In [ ]:
# reading human chr20 fasta
chr20_fasta = pysam.FastaFile('/Users/nemanjavucic/Desktop/matf-rna-seq-1/chr20.fasta')

In [ ]:
# function that returns a reverse complement of the given sequence
def revc(dna):
    """Reverse complement a DNA sequence."""
    revc = dna[::-1].translate(str.maketrans('ACGT','UGCA'))
    return(revc)

In [ ]:
def rna2prot_single(seq):
    protein = ''
    i = 0
    protein1 = ''
    while i <= len(seq)-3:
            protein1 += codon_map[seq[i:i+3]]
            i += 3
    return protein1 

In [ ]:
# codon map for translation
codon_map = {"UUU":"F", "UUC":"F", "UUA":"L", "UUG":"L",
    "UCU":"S", "UCC":"S", "UCA":"S", "UCG":"S",
    "UAU":"Y", "UAC":"Y", "UAA":"STOP", "UAG":"STOP",
    "UGU":"C", "UGC":"C", "UGA":"STOP", "UGG":"W",
    "CUU":"L", "CUC":"L", "CUA":"L", "CUG":"L",
    "CCU":"P", "CCC":"P", "CCA":"P", "CCG":"P",
    "CAU":"H", "CAC":"H", "CAA":"Q", "CAG":"Q",
    "CGU":"R", "CGC":"R", "CGA":"R", "CGG":"R",
    "AUU":"I", "AUC":"I", "AUA":"I", "AUG":"M",
    "ACU":"T", "ACC":"T", "ACA":"T", "ACG":"T",
    "AAU":"N", "AAC":"N", "AAA":"K", "AAG":"K",
    "AGU":"S", "AGC":"S", "AGA":"R", "AGG":"R",
    "GUU":"V", "GUC":"V", "GUA":"V", "GUG":"V",
    "GCU":"A", "GCC":"A", "GCA":"A", "GCG":"A",
    "GAU":"D", "GAC":"D", "GAA":"E", "GAG":"E",
    "GGU":"G", "GGC":"G", "GGA":"G", "GGG":"G",}

In [ ]:
peptides = []
for i in range(len(start)):
    peptides.append(rna2prot_single(revc(chr20_fasta.fetch('20', start[i], end[i]))))

In [ ]:
''.join(peptides)

In [ ]:
#load BAM file

In [ ]:
rna_bam = pysam.AlignmentFile("/Users/nemanjavucic/Downloads/chr20.twopass-basic.Aligned.sorted.bam")
for read in rna_bam.fetch():
    if 'M' in str(read.cigarstring) and 'N' in str(read.cigarstring):
        print(read)
        break